Google ColaboratoryでGoogle Driveに置いてある画像ファイルを扱うには、Google Driveをランタイムマシン上にマウントする必要がある。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

PIL (Python Image Library) を用いて画像ファイルを読み込み、フォーマットや解像度、チャネル情報を表示してみよう。変数myfileの値は、各自の画像ファイル名に書き換えて下さい。JPEG形式の圧縮ファイルであっても、画像を読み込むと伸張されて解像度xRGBチャネルのデータとして扱うことができる。

In [ ]:
from PIL import Image
myfile='Colab/20080405102551-5c73f95a.jpeg'
img=Image.open('/content/drive/MyDrive/'+myfile)
print(img.format, img.size, img.mode)

縦横それぞれ1/4の大きさに縮小した画像を作成し、表示してみよう。

In [ ]:
w, h=img.size
w, h=w//4, h//4
small=img.resize((w, h))
print(small.size)
small.show()

チャネル毎の画像を表示してみよう。

In [ ]:
for i in small.getbands():
  print(i)
  small.getchannel(i).show()

赤チャネルの画像を赤色で表示したい場合は、他の色を0にしたRGB画像を合成すればよい。

In [ ]:
import numpy as np
black=np.zeros((small.size[1], small.size[0]), dtype=np.uint8)
black_img=Image.fromarray(black).convert('L')
red=Image.merge("RGB", (small.getchannel('R'), black_img, black_img))
red.show()

HSV空間に変更して、輝度すなわちVのヒストグラムを表示してみよう。

In [ ]:
import matplotlib.pyplot as plt
h, s, v=small.convert('HSV').split()
v_hist=v.histogram()
plt.plot(v_hist)
plt.show()

黒潰れしている領域のコントラストを上げて見やすくしてみよう。

In [ ]:
x=list(map(lambda x: 255 if x > 63 else x*4, range(256)))
v_enhance=v.point(x, mode='L')
enhanced=Image.merge("HSV", (h, s, v_enhance)).convert("RGB")
enhanced.show()
plt.ylim((0, 6000))
plt.plot(v_hist)
plt.plot(v_enhance.histogram())
plt.show()

逆に、白飛びしている領域のコントラストを上げてみよう。

In [ ]:
x=list(map(lambda x: 0 if x < 192 else (x-192)*4, range(256)))
v_enhance=v.point(x, mode='L')
enhanced=Image.merge("HSV", (h, s, v_enhance)).convert("RGB")
enhanced.show()
plt.ylim((0, 6000))
plt.plot(v_hist)
plt.plot(v_enhance.histogram())
plt.show()

ピクセル値を書き換えてみよう。画像を表示して、効果を確認してみよう。

In [ ]:
for x in range(0, 256):
  for y in range(0, 256):
    small.putpixel((x+20, y+20), (x, 128, y))
small.show()